In [1]:
import os
# We save the model to a local mounted directory
os.environ['HF_HOME'] = '/workspace/llm_models'

MODEL_NAME="gpt-4o-2024-11-20"

modality = "passage"

In [2]:
import sys
import os
sys.path.append('/workspace/src/')

In [3]:
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/workspace/src/']


In [4]:
from dotenv import dotenv_values, load_dotenv
from umbrela.gpt_judge import GPTJudge
from umbrela.osllm_judge import OSLLMJudge
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)
from collections import defaultdict

import pickle

import json
from tqdm import tqdm

from preprocessing.utils import parallel_process_topics, single_process_topics, load_vectorstore, load_pool_documents, parallel_process_topics_new, parallel_process_topics_safe, process_topic

db_vals = dotenv_values("/workspace/src/.env")
load_dotenv()

True

In [5]:
#load missing_ids from /workspace/src/data/missing_ids/

missing_ids_bm25 = json.load(open("/workspace/src/data/missing_ids/nan_ids_bm25.json", "r"))
missing_ids_vec = json.load(open("/workspace/src/data/missing_ids/nan_ids_vectorstore.json", "r"))

all_missing_ids = list(missing_ids_bm25['BM25_Text'] + missing_ids_vec['Vectorstore_Text'] +missing_ids_bm25["BM25_Table"] + missing_ids_vec['Vectorstore_Table'])


In [6]:
#convert list to dictionary with topic number as key and docids as list
missing_ids_dict = defaultdict(list)

for item in all_missing_ids:
    missing_ids_dict[f"{item['topic']}"].append(item['docid'])

missing_pool = dict(missing_ids_dict)

In [5]:
passage_pool = json.load(open("/workspace/src/data/passage_pool.json", "r"))
table_pool = json.load(open("/workspace/src/data/table_pool.json", "r"))

passage_pool_for_rel = json.load(open("/workspace/src/data/passage_pool_for_rel.json", "r"))

topics = pickle.load(open("/workspace/src/data/topics.pkl", "rb"))
vectorstore = load_vectorstore("/workspace/src/preprocessing/vectorstores/chromadb_store")

/workspace/src/preprocessing/utils.py:87: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model=embedding_model)
/workspace/src/preprocessing/utils.py:88: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [6]:
if modality == 'table':
    pool_documents = load_pool_documents(table_pool, vectorstore)
elif modality == 'passage':
    pool_documents = load_pool_documents(passage_pool, vectorstore)

if modality == 'missing':
    pool_documents = load_pool_documents(missing_pool, vectorstore)


In [7]:
os.environ["DEPLOYMENT_NAME"] = MODEL_NAME

In [8]:
judge_gpt = GPTJudge(qrel="cord19", prompt_type="bing")

Warning!! Prompt file expects input fields namely: (examples, query, passage).


In [11]:
topics = {k:v for k,v in topics.items() if k in pool_documents.keys()}

In [9]:
all_judgments = parallel_process_topics_safe(topics, pool_documents, judge_gpt, max_workers=10, verbose=False, MODEL_NAME=MODEL_NAME)

Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).
Warning!! Prompt file expects input fields namely: (examples, query, passage).


Processing topics:   2%|▏         | 1/50 [01:53<1:32:47, 113.63s/topic]

Warning!! Prompt file expects input fields namely: (examples, query, passage).


Processing topics:   4%|▍         | 2/50 [01:56<38:34, 48.22s/topic]   

Warning!! Prompt file expects input fields namely: (examples, query, passage).


...

Processing topics:  78%|███████▊  | 39/50 [08:24<01:02,  5.68s/topic]

Warning!! Prompt file expects input fields namely: (examples, query, passage).


Processing topics:  80%|████████  | 40/50 [08:33<01:05,  6.51s/topic]

Warning!! Prompt file expects input fields namely: (examples, query, passage).


Processing topics: 100%|██████████| 50/50 [10:43<00:00, 12.86s/topic]


In [10]:
print(modality, MODEL_NAME) 

passage gpt-4o-2024-11-20


In [11]:
with open(f'/workspace/src/data/qrels_{modality}_pool_{MODEL_NAME.replace("/", "_")}.json', 'w', encoding='utf-8') as json_file:
        json.dump(all_judgments, json_file, ensure_ascii=False, indent=4) 

In [ ]:
pool_documents